<a href="https://colab.research.google.com/github/dawidstajszczyk/thesis_APO/blob/main/image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Liczba czarnych kropek: 2


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def count_black_dots(image_path):
    # Wczytanie obrazu
    image = cv2.imread(image_path)

    # Przekształcenie obrazu na odcienie szarości
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Binaryzacja obrazu (czarne kropki będą miały wartość 0)
    _, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY_INV)
    cv2_imshow(binary)

    # Rozmycie Gaussowskie w celu wygładzenia krawędzi
    blurred = cv2.GaussianBlur(binary, (5, 5), 0)
    cv2_imshow(blurred)

    # Detekcja krawędzi w obrazie
    edges = cv2.Canny(blurred, 50, 150)
    cv2_imshow(edges)

    # Zastosowanie dylatacji w celu połączenia ewentualnych przerw w krawędziach
    kernel = np.ones((5, 5), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=1)
    cv2_imshow(dilated)

    # Znalezienie konturów w obrazie
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Utworzenie kopii obrazu, na którym będą rysowane kontury czarnych kropek
    dots_image = image.copy()
    cv2.drawContours(dots_image, contours, -1, (0, 255, 0), 2)

    # Zliczenie czarnych kropek na podstawie pola powierzchni i kolistości
    black_dot_count = 0
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 100:  # Minimalna wielkość obszaru, aby uznać go za czarną kropkę
            continue

        # Sprawdzenie, czy kontur jest zbliżony do kształtu okręgu
        perimeter = cv2.arcLength(contour, True)
        circularity = 4 * np.pi * area / (perimeter * perimeter)
        if circularity > 0.6:  # Określ próg, powyżej którego uznajemy kształt za okrągły
            cv2.drawContours(dots_image, [contour], -1, (0, 255, 0), 2)
            black_dot_count += 1

    # Wyświetlenie obrazu z konturami czarnych kropek
    cv2_imshow(dots_image)

    return black_dot_count

# Ścieżka do zdjęcia
image_path = "/content/Untitled design (1).png"

# Zliczenie czarnych kropek
dots_count = count_black_dots(image_path)
print("Liczba czarnych kropek:", dots_count)
